<a href="https://colab.research.google.com/github/Andy-yun-liang/Sassafras-Regression/blob/main/sassafras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
import sklearn

In [ ]:
trainX = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Xtrain.txt",sep=" ")
trainY = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ytrain.txt")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
testX = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Xtest.txt",sep=" ")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
testX  

,Id,#A01,#A02,#B01,#B02,#B03,#B04,#B05,#B06,#B07,#B08,#B09,#B10,#B11,#B12,#B13,#B14,#B15,#B16,#B17,#B18,#B19,#B20,#B21,#B22,#B23,#B24,#C01,#C02,#C03,#D01,#D02,#D03,#D04,#D05,#D06,#D07,#D08,#D09,#D10,#D11,#D12,#D13,#D14,#D15,#D16,#D17,#E01,#F01,#F02,#F03,#F04,#F05,#F06,#F07,#F08,#F09,#F10,#F11,#F12,#G01,#G02,#G03,#H01,#H02,#H03,#H04,#H05,#H06,#H07,#H08,#H09,#I01,#I02,#I03,#I04
0,1,1.727140,1.654740,-1.097710,-0.385983,-1.244110,0.419808,-1.435640,-0.376851,0.574999,0.730206,-0.859501,0.613872,-9.000000,-0.425670,0.326276,1.283410,0.260533,-0.247223,?,0.343297,0.900679,-1.874900,0.110853,-2.147730,-1.065340,0.291275,0.485545,-0.030703,0.986083,0.064466,0.096887,-0.042067,0.090927,0.036404,-0.025910,-0.034691,-0.066579,-0.025245,-0.076142,-0.119019,0.031301,0.055641,-0.023539,-0.034314,0.047202,0.003573,0.775531,0.575229,0.630193,0.075884,0.363873,0.226764,0.857523,0.554989,0.680934,0.033421,0.807272,0.800452,0.861571,0,0,0,-1.903670,-0.755047,-0.412940,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
1,6,1.205050,0.296842,-1.265870,1.001000,-0.285456,1.296480,-0.150170,NaN,1.084120,-1.039780,0.190630,1.059380,0.534187,0.145579,-0.951917,-0.168384,-0.099459,-2.018620,0.118709,0.013412,-1.313690,-0.027979,0.777201,-1.721460,-1.998120,1.732180,-1.982470,-1.173540,-1.964330,-0.103334,-0.085467,-0.012393,-0.076087,0.122670,-0.046947,0.070825,-0.092639,0.111251,0.025333,-0.049026,0.039659,0.032205,-0.038788,0.023325,-0.048956,-0.037861,-0.831560,0.211762,0.387266,0.005525,0.340464,0.636823,0.787351,0.800078,0.222068,0.265822,0.396478,0.223732,0.199441,0,0,0,-1.849470,0.541212,-0.579240,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
2,9,-1.132310,0.307465,-0.048653,-0.210259,NaN,0.314849,0.801982,0.038014,0.515885,-0.418177,0.082589,0.940683,1.074700,1.798310,0.658646,-0.954989,0.922123,-1.790410,-0.338825,NaN,0.781843,-0.959706,0.888815,0.633445,-0.004809,0.088348,1.221110,0.691730,-0.667247,0.056826,-0.015687,0.014879,0.034132,-0.020627,0.089384,-0.096474,-0.070312,0.079820,0.035325,0.013307,0.045409,0.008459,-0.029906,0.161741,0.100794,-0.039171,1.200000,0.131386,0.761758,0.132647,0.580234,0.568945,0.584636,0.906048,0.731583,0.174916,0.246667,0.070685,0.258877,0,0,0,-2.479640,-1.804930,0.764894,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
3,12,-0.954363,-0.720769,-0.987462,0.543249,NaN,-0.387820,-0.199311,0.773217,0.440916,0.943336,0.558700,-1.727980,-0.806496,-0.330980,0.854073,1.119300,0.923518,-0.962399,0.204848,-0.679889,-0.139566,-0.076675,NaN,NaN,-0.517915,1.863240,-0.076221,0.131434,-1.376950,0.090110,-0.018302,0.042275,0.020112,0.030639,-0.039688,-0.068676,-0.056414,0.070493,-0.154153,0.027550,-0.012530,-0.049675,-0.024532,0.003731,-0.008487,-0.026318,0.141018,0.826122,0.366182,0.954950,0.729105,0.474379,0.860224,0.063835,0.826185,0.869053,0.539819,0.580451,0.511207,0,0,0,-1.766040,-1.558680,-1.311400,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
4,13,-0.535508,-0.529080,-0.681735,0.407560,-0.023905,1.623960,0.152597,NaN,0.261951,-0.277629,1.021780,-1.303790,0.677249,0.480057,-0.213977,0.090285,-0.569297,-2.056100,0.215238,0.649046,0.238653,0.059108,0.457152,0.711012,NaN,0.093264,-1.553700,-0.426444,0.365619,0.070092,0.130168,-0.003816,-0.071428,-0.019837,-0.039779,0.059929,-0.096043,0.050352,-0.054802,-0.001601,-0.036523,0.000889,-0.030000,0.041713,0.034629,-0.000489,-0.774024,0.010775,0.855917,0.906981,0.458802,0.923445,0.706043,0.532373,0.697095,0.216771,0.400600,0.741049,0.992279,0,0,0,-1.738970,0.225965,0.360930,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,177717,2.202510,2.295670,-0.813234,-0.571600,1.107120,-0.744815,-1.131610,-1.458800,0.719253,-0.175299,0.337345,-0.168789,0.282036,-1.077370,-0.326640,0.028627,0.695467,-0.012297,0.946064,0.923167,-1.118130,-0.345184,-0.462411,0.225260,0.58

In [ ]:
testX.shape #50000* 76 
trainX.shape #153287 * 76

(153287, 76)

#Data Preprocessing


In [ ]:
#chaning ? to nan
trainX["#B17"] = trainX["#B17"].replace("?",np.nan).astype('float64') 
testX["#B17"] = testX["#B17"].replace("?",np.nan).astype('float64') 

In [ ]:
#mean fill
trainX = trainX.fillna(trainX.mean())
testX = testX.fillna(testX.mean())

In [ ]:
#removing low variance features from both test and train set

In [ ]:
from sklearn.feature_selection import VarianceThreshold
the_threshold = VarianceThreshold(threshold=0.05)
the_threshold.fit(trainX)

VarianceThreshold(threshold=0.05)

In [ ]:
the_threshold.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [ ]:
to_be_remove = [i for i in testX.columns
               if i not in testX.columns[the_threshold.get_support()]]

In [ ]:
testX.drop(to_be_remove,axis=1,inplace=True)
trainX.drop(to_be_remove,axis=1,inplace=True)

In [ ]:
trainY.isnull().sum()
#no missing values for train Y

Id       0
Value    0
dtype: int64

In [ ]:
missing = trainX.isnull().sum().sort_values(ascending=False) 

In [ ]:
display(pd.DataFrame(missing[missing[missing>0 ].index],columns=["Count of NA's"]))
#no missing values for Train features

,Count of NA's


In [ ]:
missing1 = testX.isnull().sum().sort_values(ascending=False)

In [ ]:
display(pd.DataFrame(missing1[missing1[missing1>0 ].index],columns=["Count of NA's"]))
#no missing values for test features

,Count of NA's


In [ ]:
corr = trainX.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
#removing highly correlated features

In [ ]:
#from sklearn feature selection documentation
def correlation(dataset, threshold):
    col_corr = set()  
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_ft = correlation(trainX, 0.8)

In [ ]:
trainX.drop(corr_ft,axis=1,inplace=True)
testX.drop(corr_ft,axis=1,inplace=True)

In [ ]:
trainX.drop(columns="Id",inplace=True)

In [ ]:
testX.drop(columns="Id",inplace=True)

In [ ]:
testX

,#A01,#A02,#B01,#B02,#B03,#B04,#B05,#B06,#B07,#B08,#B09,#B10,#B11,#B12,#B13,#B14,#B15,#B16,#B17,#B18,#B19,#B20,#B21,#B22,#B23,#B24,#C01,#C02,#C03,#D01,#E01,#F01,#F02,#F03,#F04,#F05,#F06,#F07,#F08,#F09,#F10,#F11,#F12,#G01,#G02,#G03,#H01,#H02,#H03
0,1.727140,1.654740,-1.097710,-0.385983,-1.244110,0.419808,-1.435640,-0.376851,0.574999,0.730206,-0.859501,0.613872,-9.000000,-0.425670,0.326276,1.283410,0.260533,-0.247223,0.005377,0.343297,0.900679,-1.874900,0.110853,-2.147730,-1.065340,0.291275,0.485545,-0.030703,0.986083,0.064466,0.775531,0.575229,0.630193,0.075884,0.363873,0.226764,0.857523,0.554989,0.680934,0.033421,0.807272,0.800452,0.861571,0,0,0,-1.903670,-0.755047,-0.412940
1,1.205050,0.296842,-1.265870,1.001000,-0.285456,1.296480,-0.150170,-0.029188,1.084120,-1.039780,0.190630,1.059380,0.534187,0.145579,-0.951917,-0.168384,-0.099459,-2.018620,0.118709,0.013412,-1.313690,-0.027979,0.777201,-1.721460,-1.998120,1.732180,-1.982470,-1.173540,-1.964330,-0.103334,-0.831560,0.211762,0.387266,0.005525,0.340464,0.636823,0.787351,0.800078,0.222068,0.265822,0.396478,0.223732,0.199441,0,0,0,-1.849470,0.541212,-0.579240
2,-1.132310,0.307465,-0.048653,-0.210259,-0.037331,0.314849,0.801982,0.038014,0.515885,-0.418177,0.082589,0.940683,1.074700,1.798310,0.658646,-0.954989,0.922123,-1.790410,-0.338825,0.010754,0.781843,-0.959706,0.888815,0.633445,-0.004809,0.088348,1.221110,0.691730,-0.667247,0.056826,1.200000,0.131386,0.761758,0.132647,0.580234,0.568945,0.584636,0.906048,0.731583,0.174916,0.246667,0.070685,0.258877,0,0,0,-2.479640,-1.804930,0.764894
3,-0.954363,-0.720769,-0.987462,0.543249,-0.037331,-0.387820,-0.199311,0.773217,0.440916,0.943336,0.558700,-1.727980,-0.806496,-0.330980,0.854073,1.119300,0.923518,-0.962399,0.204848,-0.679889,-0.139566,-0.076675,0.004503,0.008599,-0.517915,1.863240,-0.076221,0.131434,-1.376950,0.090110,0.141018,0.826122,0.366182,0.954950,0.729105,0.474379,0.860224,0.063835,0.826185,0.869053,0.539819,0.580451,0.511207,0,0,0,-1.766040,-1.558680,-1.311400
4,-0.535508,-0.529080,-0.681735,0.407560,-0.023905,1.623960,0.152597,-0.029188,0.261951,-0.277629,1.021780,-1.303790,0.677249,0.480057,-0.213977,0.090285,-0.569297,-2.056100,0.215238,0.649046,0.238653,0.059108,0.457152,0.711012,0.021372,0.093264,-1.553700,-0.426444,0.365619,0.070092,-0.774024,0.010775,0.855917,0.906981,0.458802,0.923445,0.706043,0.532373,0.697095,0.216771,0.400600,0.741049,0.992279,0,0,0,-1.738970,0.225965,0.360930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2.202510,2.295670,-0.813234,-0.571600,1.107120,-0.744815,-1.131610,-1.458800,0.719253,-0.175299,0.337345,-0.168789,0.282036,-1.077370,-0.326640,0.028627,0.695467,-0.012297,0.946064,0.923167,-1.118130,-0.345184,-0.462411,0.225260,0.588991,-0.145310,1.479710,-2.201110,-1.897500,0.014988,-0.910196,0.545659,0.165264,0.982262,0.352101,0.606007,0.876506,0.911822,0.170132,0.007715,0.119904,0.254041,0.121196,0,0,0,-1.990390,1.426950,0.851668
49996,0.486308,0.846261,-1.108650,1.051280,-0.037331,0.511571,-0.219691,0.110070,-0.228894,-1.068120,1.391660,0.796395,-0.312693,0.143449,-0.064277,-0.343960,1.957830,0.011033,1.130810,0.010754,0.098900,0.287627,0.162322,1.173690,-0.755598,0.005370,-0.243006,0.233812,-0.904462,0.183358,-1.561950,0.390416,0.150480,0.113966,0.324749,0.204694,0.105963,0.164940,0.667723,0.091087,0.534935,0.245038,0.650371,0,0,0,-0.365453,0.850150,1.106730
49997,-0.184787,0.372172,2.014950,-0.135052,-0.037331,-0.022866,0.637608,0.318490,-0.219347,0.772532,0.152524,-0.299363,-0.678613,-0.683192,-0.469430,0.162422,0.807231,-0.951113,-1.490270,0.145634,0.001759,3.418620,0.985110,0.120230,-0.541069,-1.480240,0.455421,0.357320,-1.127160,0.050054,-0.118389,0.965798,0.853460,0.681459,0.414713,0.562400,0.286590,0.532012,0.636644,0.430731,0.925973,0.940587,0.303778,0,0,0,0.875423,1.233030,0.593078
49998,-0.353409,-0.674765,0.227447,-1.

In [ ]:
#adding interaction terms
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2,interaction_only=True)
poly.fit(trainX)
trainX = poly.transform(trainX)
testX = poly.fit_transform(testX)


In [ ]:
trainX = pd.DataFrame(trainX)
testX = pd.DataFrame(testX)
trainX

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225
0,1.0,-0.067006,2.000400,-0.181087,-0.288238,-0.026722,-1.332320,-0.478257,1.672440,-2.526620,-9.000000,-0.724482,-1.709480,2.187080,0.860115,0.435930,-0.294076,1.067530,1.439820,0.005673,-0.919251,-1.996300,0.890578,0.149513,-1.160060,-0.682101,-0.713485,1.542960,1.299720,1.596820,0.354335,0.068042,0.993830,0.776477,0.862845,0.603155,0.525250,0.000804,0.088207,0.864470,...,0.0,-1.300895,1.336585,0.050990,0.058683,0.107951,0.0,0.0,0.0,-0.237654,0.244174,0.009315,0.199837,0.0,0.0,0.0,-0.439942,0.452012,0.017244,0.0,0.0,0.0,-0.809308,0.831511,0.031722,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-1.830571,-0.069836,0.071752
1,1.0,-0.238565,-1.755040,-0.562340,-1.943580,0.135810,-0.027425,-0.952346,-0.030758,-0.745675,-1.729910,1.159080,0.962413,-1.962020,0.178107,-0.019330,1.030860,-0.843428,-0.100388,-0.971939,-1.585190,0.318801,-0.667907,0.008419,0.006790,1.270770,-0.826329,-0.878540,0.407246,1.306110,-0.172220,-0.900000,0.581592,0.624055,0.647532,0.455346,0.222504,0.073445,0.875287,0.766641,...,0.0,0.563648,-0.780852,0.124532,0.116806,0.234545,0.0,0.0,0.0,0.335008,-0.464106,0.074016,0.084635,0.0,0.0,0.0,0.120887,-0.167471,0.026709,0.0,0.0,0.0,0.242740,-0.336281,0.053631,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.480321,0.076602,-0.106122
2,1.0,-0.900302,0.760359,-0.566809,0.843982,0.902949,-0.298859,-0.032618,-2.648990,1.090920,-1.496180,1.458000,-0.151557,0.005473,-0.069429,0.011816,0.341582,-0.974272,1.218820,1.492320,2.009810,1.736070,-1.863180,-0.927577,0.185668,-0.031869,-0.445829,0.790598,-0.377092,-1.537980,0.059968,0.880008,0.758466,0.657927,0.671026,0.853278,0.039861,0.042891,0.448052,0.357999,...,0.0,-0.371403,-1.040359,1.230705,0.642930,0.650942,0.0,0.0,0.0,-0.340485,-0.953754,1.128254,0.699444,0.0,0.0,0.0,-0.365855,-1.024818,1.212321,0.0,0.0,0.0,-0.370414,-1.037588,1.227427,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.542726,-0.642025,-1.798415
3,1.0,0.448940,-0.530155,-0.723218,0.014280,0.175458,0.932137,0.350433,-1.143190,-0.024211,1.157120,-0.703945,-0.361253,1.116860,0.906481,0.459762,1.415540,-1.417520,1.311010,0.553186,0.118350,1.253730,-1.367460,0.008419,-0.455266,-1.841490,0.869690,-0.019017,1.323400,-0.174755,-0.095152,1.032700,0.424483,0.137043,0.421215,0.814396,0.221353,0.118081,0.559297,0.711032,...,0.0,-1.443530,0.951259,0.362254,0.773461,0.300785,0.0,0.0,0.0,-1.974123,1.300910,0.495406,0.263510,0.0,0.0,0.0,-1.729482,1.139697,0.434014,0.0,0.0,0.0,-0.672564,0.443207,0.168780,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-2.908877,-1.107744,0.729983
4,1.0,-0.096147,0.123017,-0.269223,0.229115,-0.041637,-0.070465,-0.198871,0.206065,-0.289754,1.130150,1.215680,0.222229,2.340990,0.006305,2.803530,0.266259,-0.246009,-0.642589,0.005673,-0.556802,1.606080,-0.636420,-0.817540,-0.674955,1.483620,-1.489480,0.915046,-0.520752,-1.999460,0.030626,-0.264000,0.773113,0.957418,0.691213,0.944098,0.226894,0.842596,0.989248,0.231087,...,0.0,-0.000276,0.001919,-0.001714,0.002678,0.168285,0.0,0.0,0.0,-0.031440,0.218983,-0.195578,0.010845,0.0,0.0,0.0,-0.002026,0.014112,-0.012604,0.0,0.0,0.0,-0.127341,0.886942,-0.792145,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.165704,0.147993,-1.030786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153282,1.0,-0.341901,-0.307778,0.599235,0.580111,-0.033728,-0.038976,-0.274740,-0.276337,2.676090,-0.032354,-0.621318,-0.939524,0.405925,-2.039790,0.802036,0.07615

In [ ]:
the_threshold2 = VarianceThreshold(threshold=0.05)
the_threshold2.fit(testX)

VarianceThreshold(threshold=0.05)

In [ ]:
the_threshold2.get_support()
to_be_remove2 = [i for i in testX.columns
               if i not in testX.columns[the_threshold2.get_support()]]

In [ ]:
to_be_remove2

[0]

In [ ]:
testX.drop(to_be_remove2,axis=1,inplace=True)
trainX.drop(to_be_remove2,axis=1,inplace=True)

In [ ]:
corr_ft2 = correlation(trainX, 0.7)
testX.drop(corr_ft2,axis=1,inplace=True)
trainX.drop(corr_ft2,axis=1,inplace=True)

In [ ]:
testX["mean"] = testX.mean(axis=1)
trainX["mean"] = trainX.mean(axis=1)

In [ ]:
testX

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,mean
0,1.727140,1.654740,-1.097710,-0.385983,-1.244110,0.419808,-1.435640,-0.376851,0.574999,0.730206,-0.859501,0.613872,-9.000000,-0.425670,0.326276,1.283410,0.260533,-0.247223,0.005377,0.343297,0.900679,-1.874900,0.110853,-2.147730,-1.065340,0.291275,0.485545,-0.030703,0.986083,0.064466,0.775531,0.575229,0.630193,0.075884,0.363873,0.226764,0.857523,0.554989,0.680934,0.033421,0.807272,0.800452,0.861571,0.0,0.0,0.0,-1.903670,-0.755047,-0.412940,2.857968,-1.895899,-0.666647,-2.148752,0.725067,-2.479551,-0.650874,0.993104,1.261168,-1.484479,1.060243,-0.127825
1,1.205050,0.296842,-1.265870,1.001000,-0.285456,1.296480,-0.150170,-0.029188,1.084120,-1.039780,0.190630,1.059380,0.534187,0.145579,-0.951917,-0.168384,-0.099459,-2.018620,0.118709,0.013412,-1.313690,-0.027979,0.777201,-1.721460,-1.998120,1.732180,-1.982470,-1.173540,-1.964330,-0.103334,-0.831560,0.211762,0.387266,0.005525,0.340464,0.636823,0.787351,0.800078,0.222068,0.265822,0.396478,0.223732,0.199441,0.0,0.0,0.0,-1.849470,0.541212,-0.579240,0.357709,-1.525437,1.206255,-0.343989,1.562323,-0.180962,-0.035173,1.306419,-1.252987,0.229719,1.276606,-0.041346
2,-1.132310,0.307465,-0.048653,-0.210259,-0.037331,0.314849,0.801982,0.038014,0.515885,-0.418177,0.082589,0.940683,1.074700,1.798310,0.658646,-0.954989,0.922123,-1.790410,-0.338825,0.010754,0.781843,-0.959706,0.888815,0.633445,-0.004809,0.088348,1.221110,0.691730,-0.667247,0.056826,1.200000,0.131386,0.761758,0.132647,0.580234,0.568945,0.584636,0.906048,0.731583,0.174916,0.246667,0.070685,0.258877,0.0,0.0,0.0,-2.479640,-1.804930,0.764894,-0.348146,0.055091,0.238078,0.042270,-0.356507,-0.908092,-0.043044,-0.584142,0.473506,-0.093516,-1.065145,0.091741
3,-0.954363,-0.720769,-0.987462,0.543249,-0.037331,-0.387820,-0.199311,0.773217,0.440916,0.943336,0.558700,-1.727980,-0.806496,-0.330980,0.854073,1.119300,0.923518,-0.962399,0.204848,-0.679889,-0.139566,-0.076675,0.004503,0.008599,-0.517915,1.863240,-0.076221,0.131434,-1.376950,0.090110,0.141018,0.826122,0.366182,0.954950,0.729105,0.474379,0.860224,0.063835,0.826185,0.869053,0.539819,0.580451,0.511207,0.0,0.0,0.0,-1.766040,-1.558680,-1.311400,0.687875,0.942397,-0.518457,0.035627,0.370121,0.190215,-0.737930,-0.420794,-0.900285,-0.533203,1.649120,0.039134
4,-0.535508,-0.529080,-0.681735,0.407560,-0.023905,1.623960,0.152597,-0.029188,0.261951,-0.277629,1.021780,-1.303790,0.677249,0.480057,-0.213977,0.090285,-0.569297,-2.056100,0.215238,0.649046,0.238653,0.059108,0.457152,0.711012,0.021372,0.093264,-1.553700,-0.426444,0.365619,0.070092,-0.774024,0.010775,0.855917,0.906981,0.458802,0.923445,0.706043,0.532373,0.697095,0.216771,0.400600,0.741049,0.992279,0.0,0.0,0.0,-1.738970,0.225965,0.360930,0.283327,0.365075,-0.218252,0.012801,-0.869644,-0.081717,0.015630,-0.140277,0.148673,-0.547171,0.698190,0.076305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2.202510,2.295670,-0.813234,-0.571600,1.107120,-0.744815,-1.131610,-1.458800,0.719253,-0.175299,0.337345,-0.168789,0.282036,-1.077370,-0.326640,0.028627,0.695467,-0.012297,0.946064,0.923167,-1.118130,-0.345184,-0.462411,0.225260,0.588991,-0.145310,1.479710,-2.201110,-1.897500,0.014988,-0.910196,0.545659,0.165264,0.982262,0.352101,0.606007,0.876506,0.911822,0.170132,0.007715,0.119904,0.254041,0.121196,0.0,0.0,0.0,-1.990390,1.426950,0.851668,5.056236,-1.791156,-1.258955,2.438443,-1.640462,-2.492382,-3.213022,1.584162,-0.386098,0.743006,-0.371759,0.039246
49996,0.486308,0.846261,-1.108650,1.051280,-0.037331,0.511571,-0.219691,0.110070,-0.228894,-1.068120,1.391660,0.796395,-0.312693,0.143449,-0.064277,-0.343960,1.957830,0.011033,1.130810,0.010754,0.098900,0.287627,0.1623

In [ ]:
trainX2 = trainX.copy()
testX2 = testX.copy()

In [ ]:
trainX2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,mean
0,-0.067006,2.000400,-0.181087,-0.288238,-0.026722,-1.332320,-0.478257,1.672440,-2.526620,-9.000000,-0.724482,-1.709480,2.187080,0.860115,0.435930,-0.294076,1.067530,1.439820,0.005673,-0.919251,-1.996300,0.890578,0.149513,-1.160060,-0.682101,-0.713485,1.542960,1.299720,1.596820,0.354335,0.068042,0.993830,0.776477,0.862845,0.603155,0.525250,0.000804,0.088207,0.864470,0.974599,0.178045,0.329594,0.606314,0.0,0.0,0.0,-1.334800,1.371420,0.052319,-0.134038,0.012134,0.019314,0.001791,0.089273,0.032046,-0.112063,0.169298,0.603052,0.048544,0.114545,0.020132
1,-0.238565,-1.755040,-0.562340,-1.943580,0.135810,-0.027425,-0.952346,-0.030758,-0.745675,-1.729910,1.159080,0.962413,-1.962020,0.178107,-0.019330,1.030860,-0.843428,-0.100388,-0.971939,-1.585190,0.318801,-0.667907,0.008419,0.006790,1.270770,-0.826329,-0.878540,0.407246,1.306110,-0.172220,-0.900000,0.581592,0.624055,0.647532,0.455346,0.222504,0.073445,0.875287,0.766641,0.957243,0.568945,0.205302,0.412246,0.0,0.0,0.0,0.588824,-0.815730,0.130094,0.418691,0.134155,0.463670,-0.032400,0.006543,0.227196,0.007338,0.177892,0.412696,-0.276516,-0.229598,-0.042092
2,-0.900302,0.760359,-0.566809,0.843982,0.902949,-0.298859,-0.032618,-2.648990,1.090920,-1.496180,1.458000,-0.151557,0.005473,-0.069429,0.011816,0.341582,-0.974272,1.218820,1.492320,2.009810,1.736070,-1.863180,-0.927577,0.185668,-0.031869,-0.445829,0.790598,-0.377092,-1.537980,0.059968,0.880008,0.758466,0.657927,0.671026,0.853278,0.039861,0.042891,0.448052,0.357999,0.843769,0.773529,0.831165,0.841522,0.0,0.0,0.0,-0.440171,-1.232990,1.458580,-0.684553,0.510299,-0.759839,-0.812927,0.269063,0.029366,2.384891,-0.982157,1.347014,-1.312640,0.136447,0.141594
3,0.448940,-0.530155,-0.723218,0.014280,0.175458,0.932137,0.350433,-1.143190,-0.024211,1.157120,-0.703945,-0.361253,1.116860,0.906481,0.459762,1.415540,-1.417520,1.311010,0.553186,0.118350,1.253730,-1.367460,0.008419,-0.455266,-1.841490,0.869690,-0.019017,1.323400,-0.174755,-0.095152,1.032700,0.424483,0.137043,0.421215,0.814396,0.221353,0.118081,0.559297,0.711032,0.687068,0.939611,0.823171,0.320116,0.0,0.0,0.0,-2.101000,1.384520,0.527246,-0.238008,-0.324681,0.006411,0.078770,0.418474,0.157323,-0.513224,-0.010869,0.519477,-0.316029,-0.162181,0.169899
4,-0.096147,0.123017,-0.269223,0.229115,-0.041637,-0.070465,-0.198871,0.206065,-0.289754,1.130150,1.215680,0.222229,2.340990,0.006305,2.803530,0.266259,-0.246009,-0.642589,0.005673,-0.556802,1.606080,-0.636420,-0.817540,-0.674955,1.483620,-1.489480,0.915046,-0.520752,-1.999460,0.030626,-0.264000,0.773113,0.957418,0.691213,0.944098,0.226894,0.842596,0.989248,0.231087,0.001787,0.203836,0.013136,0.825592,0.0,0.0,0.0,-0.154242,1.074310,-0.959487,-0.011828,0.025885,-0.022029,0.004003,0.006775,0.019121,-0.019812,0.027859,-0.108660,-0.116884,-0.021367,0.170232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153282,-0.341901,-0.307778,0.599235,0.580111,-0.033728,-0.038976,-0.274740,-0.276337,2.676090,-0.032354,-0.621318,-0.939524,0.405925,-2.039790,0.802036,0.076155,-0.404830,0.007081,1.051920,0.039994,-2.100820,-0.998773,-0.304869,-0.928896,-1.886500,-0.057068,-0.021963,0.110004,-1.999120,0.024535,0.323623,0.732126,0.749570,0.636168,0.194519,0.613770,0.342117,0.650611,0.881066,0.570629,0.389514,0.535839,0.524194,0.0,0.0,0.0,0.137691,-1.235490,-0.322039,0.105230,-0.204879,-0.198341,0.011532,0.013326,0.093934,0.094480,-0.914958,0.011062,0.212429,0.321224,-0.032788
153283,1.284220,0.592979,-0.621695,-0.009792,-0.382274,-0.664959,0.836545,-0.166343,-0.713656,-0.458630,-0.351781,0.522021,1.244510,0.470620,1.421030,0.869691,1.506580,-0.626331,0.320256,2.417000,-0.351682,0.206542,0.79

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.99).fit(trainX2)
trainX3=pca.transform(trainX2)
trainX3 = pd.DataFrame(trainX3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
trainX3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,-0.299826,-0.905097,-1.013084,-0.009019,0.406179,0.282362,-0.077478,-0.511264,0.290169,1.527637,-0.555872,0.152902,2.447057,-3.069970,-0.131874,0.623804,7.721748,0.197567,-2.855093,-0.730920,-1.285815,2.283423,1.354547,0.103917,-0.227592,-0.443972,-2.030149,1.619709,-0.227542,-0.940946,-1.750017,-0.335206,1.129438,1.599625,0.054134,-0.860350,-0.961415,-0.905019,-0.141335,2.555066,-0.014691,0.061610,-1.211175,1.848059,0.052821,-0.021297,-0.041665,0.291060,-0.545206,0.381795
1,-0.317501,0.288225,1.214352,0.899437,-0.904089,-0.093106,0.080402,-0.090086,-0.029743,0.359459,-0.185105,-0.095162,-0.734549,-1.923799,1.217604,-0.984822,1.312588,0.503507,-0.601014,0.636143,0.731123,0.507647,-1.208710,-0.006454,0.573198,-0.752370,-0.048660,1.157439,-1.145622,1.149383,-0.077615,-0.049557,0.743908,-2.022605,-0.224172,-0.194241,-0.058633,1.297051,0.539857,-0.152605,-0.286457,0.253796,1.493182,-1.227196,0.035549,0.011468,-0.001842,0.159909,-0.048938,0.060784
2,-0.152459,-1.074079,-0.717034,1.069058,-1.269070,-1.165905,1.518633,-0.531737,0.153869,0.291962,1.109366,2.499346,-0.163439,-0.482575,-0.851266,-0.666726,1.400128,0.339636,0.336628,1.833529,-0.407742,-2.153487,1.058278,-1.521188,0.965826,1.356751,-0.211885,-1.408378,2.026549,-0.437146,0.888764,1.068327,-1.880249,-0.635541,1.378503,1.098359,0.289031,0.802876,0.525800,0.304219,-0.195037,-1.502985,0.560395,0.573263,-0.026148,-0.021172,-0.017198,0.299156,-0.039430,0.518888
3,-0.222579,-0.774646,-0.200613,0.306080,-0.056819,-0.040691,-0.089352,-0.294670,-0.348354,0.354465,-0.124064,-0.218115,0.370324,-0.050122,-0.366868,-0.078128,-1.316785,-1.237626,0.233801,0.003175,0.539724,-1.064453,0.804963,1.195326,-0.836934,0.397445,-2.555387,-0.337560,1.924209,0.063908,0.043001,0.806561,-0.179047,-1.736707,0.329289,0.194835,-0.420178,0.183752,-0.079271,0.650392,1.618632,-2.013982,-1.343408,0.057699,-0.043898,-0.026918,0.009128,-0.000527,0.128665,0.411481
4,-0.177521,-0.568146,-0.218344,-0.001265,-0.020416,-0.093715,-0.025013,0.143796,-0.035819,-0.155845,-0.011028,-0.080968,-0.808029,0.470197,-0.509686,-1.066456,-0.733509,0.488101,0.065297,0.081764,0.410288,0.224427,0.361526,0.818834,-1.116669,0.101856,0.658037,-2.112083,0.287467,-1.651545,-2.084983,1.822218,1.698452,-0.281470,1.109812,1.726885,0.979968,-0.425646,0.296427,0.800886,0.458555,-0.411221,1.649815,0.573256,-0.013730,0.004865,0.024190,0.158274,0.249727,-0.351833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153282,-0.743310,-0.107806,0.729439,-0.215919,0.169192,-0.259330,0.096742,-0.125433,-0.586574,0.095010,0.072631,0.747582,0.870626,1.214919,0.177696,0.834870,-0.470641,-0.158707,0.390290,1.698485,-1.762460,-0.537529,0.065551,-1.198466,-0.081102,-0.053854,0.223873,-1.792598,0.093788,0.497773,-0.758257,-0.898274,-0.950878,0.148439,-0.843745,0.948988,1.284311,-0.948962,-1.466438,1.570030,-0.974146,-1.593270,-1.200105,-0.491859,0.093295,-0.013576,-0.042638,0.353156,-0.032392,-0.137093
153283,-0.404268,0.129804,-1.269908,-1.114689,0.079776,-0.566753,-0.344010,1.278218,-0.846350,0.491306,-0.222665,0.377001,-0.352773,-0.884569,-0.463847,0.106625,0.246587,0.212246,-0.016144,-1.318244,-0.351591,-0.436002,-0.218482,0.715668,-0.662239,-0.829181,1.529115,1.477856,1.376067,-2.938134,-0.389764,-0.017970,0.301547,0.761332,0.249664,-1.038012,1.083142,-0.004068,-1.137942,-0.726116,-0.470575,0.268702,0.680612,-0.332838,-0.028387,-0.002255,-0.006048,0.330152,-0.172454,-0.210443
153284,-0.049835,-0.315575,-0.948363,-0.917237,0.506293,-0.013090,-0.499984,-0.130314,0.058718,0.139377,-0.357775,0.410905,-1.333975,-0.850085,0.160210,-0.028169,0.222107,0.578140,0.472539,0.025236,1.179497,-0.074212,1.464297,0.921098,0.032088,0.978822,-1.889933,0.268191,-0.244528,-0.152314

In [ ]:
testX3 = pca.transform(testX2)
testX3 = pd.DataFrame(testX3)
testX3

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,-1.431931,2.452984,-3.169193,-1.841177,-0.918092,-0.103156,-0.202113,-2.495435,0.798774,1.696832,0.474031,-0.840534,-0.091668,-1.253260,-0.166709,1.218966,-1.849166,-0.117647,-0.126668,1.258567,0.417149,0.603846,0.807309,-1.259795,-1.230402,0.465839,-1.154571,1.156515,-1.062006,3.069822,4.867010,1.623934,0.383871,-3.331319,-1.701283,0.329665,0.303760,3.779076,-1.506759,-0.923928,-4.742064,-0.665463,0.077857,-0.525340,0.174414,-0.047157,-0.039573,0.384452,0.147042,-0.513349
1,-0.025459,-0.412921,-0.919756,-1.792377,-0.758626,0.181340,-0.723052,-0.579595,-0.916133,-0.493354,0.754989,-1.689828,-1.306530,-0.881930,-0.921365,0.588850,0.327179,-1.266309,-1.246029,1.154273,-0.871093,-0.263019,-2.129260,-0.044611,-2.139964,1.643847,-0.048398,-1.702979,0.481400,1.808669,0.248309,-0.561546,0.653320,0.995950,-0.523863,-0.659908,0.292093,-1.079817,-2.596040,-0.540220,0.741648,-0.698411,-1.735788,-0.476403,-0.000417,-0.022770,-0.018439,-0.458696,0.400622,-0.464868
2,-0.943656,-0.430189,0.712337,0.957335,-0.281767,0.827757,-0.161742,-0.381781,0.463332,0.272339,0.145860,0.710143,-1.065054,-0.044380,0.508787,0.037377,-0.125670,-0.256223,-0.289833,-0.310702,-0.834009,-0.459398,1.669429,-2.403580,-0.405015,1.186518,-1.939645,-0.667618,-0.061463,-1.117538,0.340428,0.352918,1.965318,1.099593,1.031320,-0.524809,0.471387,-0.093844,0.079508,-1.425434,1.089150,-1.106065,0.328438,0.209555,0.015650,-0.028063,-0.025239,-0.174300,0.400929,-0.324941
3,-0.254450,0.534086,1.145199,-0.576930,-0.129942,-1.740378,-0.056938,-0.490565,-0.233622,-1.202504,-1.300349,0.019475,0.995117,0.697678,-1.289253,-1.119823,-0.843059,-0.049670,0.211628,0.321426,-0.709125,0.625880,-0.095190,-1.785547,-1.655263,-0.042267,-0.529185,-1.297208,0.748099,-0.104495,0.201862,0.009881,1.172000,-0.406905,-2.161629,0.018300,0.141874,0.159394,-0.801845,-1.022005,-0.494911,-0.423079,-0.375944,-0.278084,0.068157,-0.022490,-0.034827,0.537184,-0.559705,0.149954
4,-0.436814,0.035155,0.544783,0.050745,-0.243017,-1.117650,-0.248164,0.258791,0.768375,0.102245,-0.224152,0.564773,0.415084,0.573699,-0.637635,-1.077060,-0.067001,-0.965993,-1.375650,0.702586,-0.042725,-0.411656,-1.568428,-0.013445,-0.952617,1.342628,-0.743707,0.614587,0.697297,0.126222,-0.263900,-0.379371,0.448005,0.284431,0.818627,-0.163225,1.127042,-0.176073,-0.346877,-1.772090,0.937546,0.078752,-0.021130,0.088010,0.029299,-0.024475,-0.021220,0.462764,-0.059608,0.219435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-0.747010,4.101999,-2.690016,-1.109338,4.104009,1.091837,-3.357803,-0.632475,1.760477,0.972698,0.147671,-2.104726,0.245993,-1.204283,-0.079413,-0.691264,0.313542,0.073089,1.553166,1.695386,0.308837,-0.886573,0.560340,0.539786,-1.295128,3.297340,0.161852,-1.333567,0.828916,-0.434342,-0.855284,-0.298268,-0.838532,1.002196,-0.242503,-0.164669,-0.005198,-0.693264,0.102043,0.203729,-1.874661,-0.182959,0.695339,0.366055,0.104284,0.003169,-0.039659,-0.131007,0.262179,-0.276283
49996,0.116302,-0.384740,-0.650823,-1.036532,0.249904,0.518077,-0.359262,0.129679,-0.295854,-0.043059,0.207264,-0.003032,-1.429617,-0.579228,-1.166389,-0.641028,0.936785,-0.130797,-0.986593,0.567584,-0.079750,0.035009,-1.372949,0.447883,0.444049,0.870215,-0.853225,-0.858423,0.239047,-0.892297,0.363466,-0.778459,0.403736,1.157564,-0.117025,0.303995,-0.697889,-0.125172,0.864730,-0.879357,-1.584470,-0.400197,-0.825887,0.398333,-0.051441,0.001931,0.006093,-0.720934,-0.307677,-0.073095
49997,-0.220230,-0.628178,-0.245506,-0.305897,0.027639,0.073403,-0.230968,-0.079456,-0.198628,0.027556,0.247605,-0.253644,0.091071,1.839340,1.036509,0.012489,-0.128497,0.583233,0.009742,-0.719030,0.327709,-0.161434,0.273770,0.948633,0.655408,0.016609,0.154819,-1.193170,-1.869299

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = StandardScaler()
scaler.fit(trainX2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


StandardScaler()

In [ ]:
trainX3 = scaler.transform(trainX2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
trainX3=pd.DataFrame(trainX3)

In [ ]:
trainX3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,-0.032649,1.726474,-1.290520e-01,-0.223406,6.220164e-03,-1.158332e+00,-3.983640e-01,1.522367e+00,-2.229837,-7.924044e+00,-0.608081,-1.470943,2.243100e+00,8.779617e-01,4.356690e-01,-0.311527,1.090927,1.476302e+00,-4.177625e-17,-9.533052e-01,-2.063954,0.908470,1.450460e-01,-1.202771e+00,-7.373572e-01,-7.386058e-01,1.505668,1.270541,1.572740,1.421957,0.061687,1.384006,0.766452,1.011461,0.276579,0.055761,-1.439581,-1.189110,1.018229,1.333592,-0.934043,-0.499863,0.284659,-0.011188,-0.009217,-0.009998,-1.311571,1.334559,0.044586,-0.294221,-0.018182,-0.018097,-0.021833,0.021789,-0.009465,-0.081197,0.063331,0.281335,-0.000253,0.034333,-0.205908
1,-0.179911,-1.462809,-4.653365e-01,-1.681444,1.505199e-01,3.880504e-16,-8.221605e-01,1.023360e-16,-0.638653,-1.500004e+00,1.058087,0.875510,-2.022951e+00,1.766617e-01,-3.199448e-02,1.047791,-0.873960,-1.107363e-01,-1.001839e+00,-1.636944e+00,0.317876,-0.722890,1.373154e-16,-3.844471e-17,1.313091e+00,-8.545681e-01,-0.864718,0.394966,1.283007,-0.782282,-0.887013,0.210942,0.332783,0.398151,-0.144047,-0.806620,-1.233093,1.047255,0.739738,1.284198,0.179217,-0.853113,-0.267398,-0.011188,-0.009217,-0.009998,0.570834,-0.804635,0.120682,-0.048978,0.043756,0.205771,-0.039204,-0.020365,0.090211,-0.020255,0.067761,0.184651,-0.164428,-0.140131,-0.340412
2,-0.747931,0.673378,-4.692783e-01,0.773862,8.316022e-01,-2.409470e-01,6.202798e-18,-2.340252e+00,1.002252,-1.293475e+00,1.322507,-0.102777,-9.364008e-17,-7.787648e-02,1.514689e-16,0.340626,-1.008497,1.248583e+00,1.523489e+00,2.053609e+00,1.775995,-1.974053,-9.622153e-01,1.843848e-01,-5.463521e-02,-4.635537e-01,0.769187,-0.374521,-1.551530,0.189692,0.857428,0.714255,0.429155,0.465073,0.988362,-1.326885,-1.319944,-0.166667,-0.423550,0.961254,0.761861,0.925650,0.953745,-0.011188,-0.009217,-0.009998,-0.436112,-1.212746,1.420498,-0.538480,0.234688,-0.410634,-0.435773,0.113398,-0.010833,1.193236,-0.530315,0.659199,-0.687733,0.045436,0.056648
3,0.410228,-0.422583,-6.072390e-01,0.043054,1.857202e-01,8.517859e-01,3.424164e-01,-9.943243e-01,0.005939,1.051050e+00,-0.589915,-0.286932,1.142714e+00,9.256393e-01,4.601504e-01,1.442454,-1.464253,1.343576e+00,5.610812e-01,1.118747e-01,1.279752,-1.455154,1.373154e-16,-4.762800e-01,-1.954677e+00,8.883172e-01,-0.023338,1.293772,-0.192884,-0.459663,1.007069,-0.236127,-1.052858,-0.246503,0.877714,-0.809898,-1.106209,0.149418,0.581435,0.515288,1.234852,0.902931,-0.529476,-0.011188,-0.009217,-0.009998,-2.061353,1.347372,0.509263,-0.340351,-0.189149,-0.024597,0.017279,0.189528,0.054522,-0.285947,-0.029557,0.238886,-0.184384,-0.105954,0.117832
4,-0.057663,0.132119,-2.067924e-01,0.232282,-7.021690e-03,-3.820556e-02,-1.486160e-01,2.116788e-01,-0.231310,1.027218e+00,1.108155,0.225481,2.401349e+00,6.689226e-17,2.867776e+00,0.263349,-0.259681,-6.694231e-01,-4.177625e-17,-5.812224e-01,1.642258,-0.689930,-8.490959e-01,-7.027320e-01,1.536577e+00,-1.536044e+00,0.891008,-0.515461,-2.011460,0.066862,-0.263721,0.755934,1.281263,0.522575,1.246812,-0.794115,0.953306,1.371057,-0.784832,-1.435002,-0.860592,-1.399268,0.908429,-0.011188,-0.009217,-0.009998,-0.156310,1.043964,-0.945385,-0.239997,-0.011202,-0.038925,-0.020708,-0.020247,-0.016067,-0.034112,-0.009590,-0.080150,-0.083804,-0.034568,0.118552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153282,-0.268613,-0.233731,5.592317e-01,0.541442,3.696298e-17,-1.025344e-02,-2.164366e-01,-2.195061e-01,2.418521,3.617518e-16,-0.516824,-0.794769,4.117396e-01,-2.103973e+00,8.117499e-01,0.068311,-0.422984,3.753684e-17,1.072175e+00,3.143619e-02,-2.171486,-1.069227,-3.220641e-01,-9.644904e-01,-2.001935e+00,-6.405042e-02,-0.026222,0.103352,

In [ ]:
testX3=scaler.fit_transform(testX2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
testX3 = pd.DataFrame(testX3)
testX3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,1.495804,1.416021,-0.949284,-0.315764,-1.070782e+00,3.965630e-01,-1.214981,-3.079836e-01,0.542311,6.758419e-01,-0.733434,0.500197,-4.704202,-5.228196e-02,0.342310,8.518288e-01,0.312758,-2.646382e-01,-2.573150e-17,3.407603e-01,9.212648e-01,-1.988348,1.085665e-01,-2.208288e+00,-1.139336e+00,2.952453e-01,0.464992,-0.037398,0.954887,0.192308,0.753707,0.190661,0.339575,-1.199940,-0.398187,-0.789312,0.979589,0.137244,0.482820,-1.322229,0.836072,0.817996,0.991056,-0.012549,-0.008264,-0.014168,-1.863220,-0.743213,-0.409063,1.012997,-0.927644,-0.351471,-1.050999,0.327255,-1.237661,-0.329496,0.453231,0.591014,-0.747026,0.360668,-0.424827
1,1.050848,0.277151,-1.099071,0.907745,-2.201628e-01,1.181918e+00,-0.097063,9.835108e-17,0.995070,-9.004414e-01,0.195004,0.729020,0.458588,2.524827e-01,-0.353433,9.190348e-02,0.125834,-2.079810e+00,1.159360e-01,2.723637e-03,-1.348148e+00,-0.054276,7.887990e-01,-1.771746e+00,-2.117286e+00,1.783226e+00,-1.937183,-1.156709,-1.966409,-0.474721,-0.808741,-0.820996,-0.337704,-1.396154,-0.463225,0.356345,0.784305,0.819604,-0.794292,-0.673762,-0.309819,-0.788986,-0.856368,-0.012549,-0.008264,-0.014168,-1.810476,0.520470,-0.570912,-0.076304,-0.751339,0.561186,-0.189627,0.735720,-0.115283,-0.038837,0.603729,-0.641393,0.086864,0.439797,-0.251235
2,-0.941185,0.286061,-0.014847,-0.160752,-2.462767e-17,3.025369e-01,0.730983,5.953197e-02,0.489741,-3.468650e-01,0.099483,0.668055,0.751277,1.134224e+00,0.523225,-3.198361e-01,0.656284,-1.845960e+00,-3.521118e-01,-1.244313e-16,7.994748e-01,-1.029969,9.027386e-01,6.399025e-01,-2.744872e-02,8.568895e-02,1.180934,0.670164,-0.682127,0.161941,1.166384,-1.044711,0.706378,-1.041641,0.202944,0.166702,0.220164,1.114638,0.623785,-0.927416,-0.727710,-1.215438,-0.690534,-0.012549,-0.008264,-0.014168,-2.423719,-1.766711,0.737248,-0.383827,0.000840,0.089398,-0.005274,-0.200403,-0.470333,-0.042553,-0.304385,0.204912,-0.070377,-0.416637,0.015915
3,-0.789528,-0.576319,-0.851082,0.503946,-2.462767e-17,-3.269394e-01,-0.139798,7.108245e-01,0.423071,8.656475e-01,0.520420,-0.702633,-0.267396,-1.764303e-03,0.629600,7.659273e-01,0.657008,-9.974875e-01,2.040544e-01,-7.077097e-01,-1.448380e-01,-0.105270,-6.286586e-17,2.309481e-17,-5.654020e-01,1.918568e+00,-0.081787,0.121402,-1.384826,0.294248,0.136820,0.888985,-0.396486,1.251571,0.616562,-0.097505,0.987106,-1.230193,0.887081,1.009431,0.090024,0.204982,0.013498,-0.012549,-0.008264,-0.014168,-1.729287,-1.526650,-1.283477,0.067542,0.423113,-0.279259,-0.008445,0.154090,0.065959,-0.370593,-0.225922,-0.468503,-0.284267,0.576035,-0.089686
4,-0.432556,-0.415550,-0.578758,0.384250,1.191268e-02,1.475286e+00,0.166241,9.835108e-17,0.263918,-2.216982e-01,0.929836,-0.484759,0.536057,4.309286e-01,0.048241,2.273006e-01,-0.118126,-2.118216e+00,2.146831e-01,6.540647e-01,2.427826e-01,0.036921,4.620812e-01,7.193386e-01,-1.855103e-16,9.076483e-02,-1.519851,-0.424992,0.340546,0.214673,-0.752804,-1.380414,0.968893,1.117796,-0.134439,1.157133,0.558031,0.074278,0.527799,-0.810629,-0.298321,0.652474,1.355747,-0.012549,-0.008264,-0.014168,-1.702944,0.213146,0.344095,-0.108710,0.148363,-0.132970,-0.019339,-0.450743,-0.066822,-0.014854,-0.091178,0.045683,-0.291062,0.228257,-0.015070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1.900941,1.953569,-0.695890,-0.479504,1.015477e+00,-6.467485e-01,-0.950579,-1.266447e+00,0.670595,-1.305670e-01,0.324717,0.098204,0.322048,-3.999676e-01,-0.013083,1.950269e-01,0.538594,-2.390671e-02,9.623029e-01,9.349596e-01,-1.147726e+00,-0.386449,-4.766424e-01,2.218819e-01,5.951063e-01,-1.556032e-01,1.432635,-2.163126,-1.900239,-0.004374,-0.885193,0.108357,-0.956645,1.32773

In [ ]:
trainY_final = trainY["Value"].copy()

In [ ]:
from sklearn.linear_model import LinearRegression , Ridge , Lasso , ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score,KFold,train_test_split,GridSearchCV,RandomizedSearchCV
import matplotlib.pyplot as plt

In [ ]:
#cv function

In [ ]:
def the_rmse(model):
    kf = KFold(3, shuffle=True, random_state=12).get_n_splits(trainX3)
    rmse= np.sqrt(-cross_val_score(model, trainX3, trainY_final, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
#linear model
ls=LinearRegression()
the_rmse(ls).mean()

In [ ]:
#lasso model
lasso_alpha = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6]
lasso_rmse = []

for i in lasso_alpha:
    lasso = make_pipeline(RobustScaler(), Lasso(alpha = i,random_state = 12))
    lasso_rmse.append(the_rmse(lasso).mean())

In [ ]:
lasso_rmse_results = pd.DataFrame(lasso_rmse,lasso_alpha,columns=['RMSE'])
display(lasso_rmse_results.transpose())

,0.001,0.005,0.010,0.050,0.100,0.150,0.200,0.250,0.300,0.350,0.400,0.450,0.500,0.600
RMSE,9.866379,9.866863,9.86766,9.878996,9.902028,9.923169,9.944934,9.971184,9.999595,10.030535,10.065218,10.103608,10.144682,10.227978


In [ ]:
#ridge model
ridge_alpha = list(range(1,16))
ridge_rmse = []

for val in ridge_alpha:
    ridge = make_pipeline(RobustScaler(), Ridge(alpha = val,random_state = 12))
    ridge_rmse.append(the_rmse(ridge).mean())

In [ ]:
ridge_rmse_results = pd.DataFrame(ridge_rmse,ridge_alpha,columns=['RMSE'])
display(ridge_rmse_results.transpose())

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
RMSE,9.866277,9.866278,9.86628,9.866281,9.866283,9.866284,9.866286,9.866287,9.866288,9.86629,9.866291,9.866293,9.866294,9.866296,9.866297


In [ ]:
#enet
enet_alpha = [0.001,0.01,0.1,0.25,0.5,0.75,1]
enet_ratio = [0.001,0.01,0.03,0.05,0.1,0.15,0.25,0.4]
enet_rmse = []

for a in enet_alpha:
    for r in enet_ratio:
        enet = make_pipeline(RobustScaler(), ElasticNet(alpha = a,l1_ratio=r,random_state = 12))
        enet_rmse.append(the_rmse(enet).mean())

In [ ]:
enet_rmse_results = pd.DataFrame(ridge_rmse,ridge_alpha,columns=['RMSE'])
display(ridge_rmse_results.transpose())

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
RMSE,9.866277,9.866278,9.86628,9.866281,9.866283,9.866284,9.866286,9.866287,9.866288,9.86629,9.866291,9.866293,9.866294,9.866296,9.866297


In [ ]:
def get_rmse(grid):
    best_rmse = np.sqrt(-grid.best_score_)
    print(best_rmse, '\n')    
    print(grid.best_params_, '\n')
    print(grid.best_estimator_)
    return best_rmse

In [ ]:
#regression tree
param_grid = {'max_depth' : [4,5,6,7,8,9,10,15] ,
              'max_features' : [4,5,6,7,8,9,10,15]
             }
tree_mod = DecisionTreeRegressor(random_state=12)
tree_grid = GridSearchCV(tree_mod, param_grid, cv=10, refit=True, verbose=1, scoring = 'neg_mean_squared_error')
tree_grid.fit(trainX,trainY_final)


In [ ]:
#regression tree score
get_rmse(tree_grid)

In [ ]:
#random_forest
rf_params = {
 'max_features':[5,10,15,20,25,35,40,45]
 }

rf_grid = GridSearchCV(RandomForestRegressor(random_state=12), rf_params,refit=True, verbose=1, scoring ='neg_mean_squared_error')
rf_grid.fit(trainX,trainY_final)
# RMSE of 8.1355 with max feature = 45

In [ ]:
rf_mod = RandomForestRegressor(max_features=45,bootstrap=True).fit(trainX,trainY_final)

the_rmse(rf_mod).mean()


In [ ]:
#gbm
#GridSearch for GradientBoostingRegressor

gbm_params ={'learning_rate' : [0.05, 0.1, 0.3, 0.5, 0.6, 0.75, 0.9, 1.1, 1.2],
            'n_estimators' : [800, 1200, 1500],
            'max_depth' : [3, 5, 7, 9, 12],
            'max_features' : [5, 8, 10, 12, 15]}

gbm_mod = GridSearchCV(GradientBoostingRegressor(random_state=12), gbm_params, scoring = 'neg_mean_squared_error', verbose = 1)
gbm_mod.fit(trainX,trainY_final)
#the rmse is 7.38905 for max_depth =3, learning_rate = 0.9


In [ ]:
gbm_mod = GradientBoostingRegressor(max_depth=3,learning_rate=0.9).fit(trainX3,trainY_final)
the_rmse(gbm_mod).mean()
#the rmse is 7.38905 for max_depth =3, learning_rate = 0.9

7.002979107340408

In [ ]:
#xgb
xgb_params = {'n_estimators' : [500, 700, 1000],
              'max_depth' : [3, 4, 5, 7],
              'learning_rate' : [0.01, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
              'reg_lambda' : [1.3, 1.5, 1.6, 1.7]
              }

xgb_mod = GridSearchCV(XGBRegressor(metric="rmse"), xgb_params, scoring = 'neg_mean_squared_error', verbose = 1, cv = 3)
xgb_mod.fit(trainX, trainY_final)
#rmse is 8.07 for max_depth =5, learning_rate = 0.5, reg_lambda =1.5


In [ ]:
xgb_final_mod = XGBRegressor(max_depth=5,learning_rate=0.5,reg_lambda=1.5,single_precision_histogram=True,metric="rmse").fit(trainX3,trainY_final)
the_rmse(xgb_final_mod).mean()
#rmse of 6.8486

[05:49:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:51:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:52:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:53:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


6.907766152222659

In [ ]:
#light gbm

lgbm_params = {'n_estimators' : [500, 700],
               'max_depth' : [3, 5, 7, 9],
               'reg_lambda_l2' : [1, 1.2, 1.6]
               'num_leaves' : [20, 30, 40, 50, 70]
               'learning_rate' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6 ]}

lgbm_mod = GridSearchCV(LGBMRegressor(objective = 'regression', metric = 'rmse'), lgbm_params, scoring = 'neg_mean_squared_error',verbose=1, cv = 10)
lgbm_mod.fit(trainX,trainY_final)



In [ ]:
lgbm_final_mod = LGBMRegressor(max_depth=9,reg_lambda_l2 = 1,learning_rate=0.4,num_leaves=20,metric="rmse",objective="regression").fit(trainX3,trainY_final)
the_rmse(lgbm_final_mod).mean()
#rmse of 6.91177

6.965201652682713

In [ ]:
#xgb_preds
xgb_pred_vals=xgb_final_mod.predict(testX3)

In [ ]:
preds['prediction'] = xgb_pred_vals
preds.to_csv('xgb.csv', index = False)

In [ ]:
#lgbm_preds 
lgbm_pred_vals =lgbm_final_mod.predict(testX3)

In [ ]:
#gbm_preds
gbm_pred_vals = gbm_mod.predict(testX3)

In [ ]:
predictions_data = np.column_stack((xgb_pred_vals,lgbm_pred_vals,gbm_pred_vals))
predictions_data = pd.DataFrame(predictions_data)

In [ ]:
preds = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/pred.txt",sep=",")

In [ ]:
#average ensemble
the_weights = [1/3, 1/3, 1/3]
preds['prediction'] = (predictions_data[0]*the_weights[0]) + (predictions_data[1]*the_weights[1]) + (predictions_data[2]*the_weights[2])
preds.to_csv('average_ensemble123.csv', index = False)
#rmse is 8.37095

In [ ]:
#weighted average ensemble
weighted_weights = [0.8,0.1,0.1]
preds['prediction'] = (predictions_data[0]*weighted_weights[0]) + (predictions_data[1]*weighted_weights[1]) + (predictions_data[2]*weighted_weights[2])
preds.to_csv('weighted_average_ensemble2.csv', index = False)
#rmse is 8.37773

In [ ]:
from mlxtend.regressor import StackingRegressor

In [ ]:
#random forest stack
random_forest_mod = RandomForestRegressor(max_depth = 45,bootstrap=True)

In [ ]:
the_regressors = StackingRegressor(regressors = [xgb_final_mod,lgbm_final_mod,gbm_mod], meta_regressor = random_forest_mod)

In [ ]:
the_regressors.fit(trainX3,trainY_final)
stacked_preds = the_regressors.predict(testX3)

[06:53:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
preds['prediction'] =stacked_preds
preds.to_csv('stacked_ensemble_withoutg.csv', index = False)
#rmse = 8.667

In [ ]:
#linear regression stack
lm_mod = LinearRegression()
the_regressors2 = StackingRegressor(regressors = [xgb_final_mod,lgbm_final_mod,gbm_mod], meta_regressor = lm_mod)
the_regressors2.fit(trainX3,trainY_final)
stacked_preds2 = the_regressors2.predict(testX3)
preds['prediction'] =stacked_preds2
preds.to_csv('linear_stack2.csv', index = False)
#rmse = 8.39391

[07:30:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
